In [15]:
import kcounter as kc
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from collections import defaultdict as dd
import sys
import numpy as np
# import matplotlib.pyplot as plt
import json

In [16]:
clusters_df = pd.read_csv('mmseqs_REPEAT_REGIONS_1_cluster_FILTERED.tsv', sep = '\t', names=['0','1'])
sequences = open('Alongiglumis_CN58138_pseudomolecules.v1.fasta.pass.list.REPEAT_REGIONS.gff3.fas')
seq_dict = {}
n = 0
for line in sequences.readlines():
    n+=1
    if n%10000==0:
        print(n)
    if '>' in line:
        name = line.strip()[1:]
    else:
        seq = line.strip()
        seq_dict[name]=seq
sequences.close()

len_df = 0
file = open('out_treetable_v6.txt')
for line in file.readlines():
    if len(line.strip().split('\t')) > len_df:
        len_df = len(line.strip().split('\t'))
last_column = len_df
file.close()
df = pd.read_csv('out_treetable_v6.txt', sep = '\t', names = [i for i in range(0,last_column)])
df = df.fillna(0)
df['cluster_full_name'] = df[df.columns[0:2]].apply(
    lambda x: ':'.join(x.dropna().astype(str)),
    axis=1)
all_clusters_names = (df['cluster_full_name'].tolist())
df = df.drop(columns=['cluster_full_name'])
rev_cols = list(df.columns)
print('rev_cols',rev_cols)
rev_cols.reverse()
for col in rev_cols:
    if len(set(df[col].values)) == 1:
        diff_column = int(col)+1
        # print('diff_column',diff_column)
        break

10000
20000
30000
40000
50000
60000
70000
80000
90000
rev_cols [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [56]:
def make_groups(df, column): #создает словарь, где ключ - клада, значение - список строк, которые в нее входят 
    grps = {}
    for i, row in df.iterrows():
        if row[column] not in grps:
            grps[row[column]] = []
        grps[row[column]].append(i)
    return(grps)

def make_tree(df,groups, column, last_col=last_column):
    node = {}
    for k, v in groups.items():
        for col in (range(column, last_col)): # recording the name of specific clade in which we look for kmers
            if (df.iloc[v,col].unique()[0]) == 0:
                needed_column=col
                break
            elif col==last_col-1:
                needed_column=col
                break
            elif len(df.iloc[v,col].unique()) == 1:
                continue
            else:
                needed_column = col
                break
        subgroups = make_groups(df.iloc[v], needed_column)
        if len(subgroups) == 1 or int(column) == last_col:
            for key, v in subgroups.items(): #k is taken from previous group index
                node[k] = v
        else:
            add_node = make_tree(df, subgroups, column + 1)
            node[k] = add_node
    return(node)

def get_leaves(branch):
    global leaves
    if type(branch) is list:
        leaves.extend(branch)
    else:
        for k, v in branch.items():
            get_leaves(v)

def make_matrix(group_item):
    max_kmer = 0
    rows = group_item[1]
    clade_name = group_item[0]
    print(df.shape[0], len(rows))
    # o=input()
    df_temp = df.iloc[rows]
    # return(df_temp)
    df_temp['cluster_full_name'] = df_temp[df_temp.columns[0:2]].apply(
    lambda x: ':'.join(x.dropna().astype(str)),
    axis=1)
    clusters_names = (df_temp['cluster_full_name'].tolist())
    # print('103\n',clusters_names)
    cluster_temp = clusters_df[clusters_df.iloc[:,0].isin(clusters_names)]
    cluster_temp_listed = cluster_temp.groupby('0')['1'].apply(list).reset_index(name='1')
    cluster_seq_dict = {}
    print('cluster_temp', cluster_temp.shape[0])
    print('cluster_temp_listed', cluster_temp_listed.shape[0])
    clusters_sizes = {}
    for i, row in cluster_temp_listed.iterrows():
        clusters_sizes[row[0]] = len(row[1])
        cluster_seq_dict[row[0]] = row[1]
    all_dicts = []
    cluster_names = []
    n = 0
    len_clusters = len(cluster_seq_dict)
    print('cluster_seq_dict len', len(cluster_seq_dict))
    # print('len cluster_names', len(cluster_names))
    for cl in clusters_names: # поменять словарь откуда кластеры брать на тот, который соответствует порядку кластеров в таблице
        cluster_kmers_amount_dict = dd(int)
        seqs_in_cluster_list = []
        n+=1
        if n%500==0:
            print(f'{n} of {len_clusters}')
        cluster_names.append(cl)
        for i in cluster_seq_dict[cl]:
            seqs_in_cluster_list.append(seq_dict[i])
        for seqq in seqs_in_cluster_list:
            kmers = kc.count_kmers(seqq, 13, canonical_kmers=True)
            for key in kmers.keys():
                cluster_kmers_amount_dict[key]+=1
        all_dicts.append(cluster_kmers_amount_dict)

    dictvectorizer = DictVectorizer(sparse=False, dtype=np.int16)
    features = dictvectorizer.fit_transform(all_dicts)
    feature_name=dictvectorizer.get_feature_names_out()
    print('first amount of kmers ',len(feature_name))

    sum_arr = np.sum(features, axis=0)
    sum_list = sum_arr.tolist()
    columns_to_del = np.where(sum_arr < 4)[0]
    features = np.delete(features, columns_to_del, axis=1)

    feature_name = np.delete(feature_name, columns_to_del)
    sum_arr = np.delete(sum_arr, columns_to_del)
    print(features)
    print('kmers after filtering ', len(feature_name))
    return(features, feature_name, clusters_names, sum_arr, clusters_sizes)

def make_csv(candidate_kmers, specificity_arr, sensitivity_arr, absolute_counts, cluster_name):
    quality_arr = specificity_arr*sensitivity_arr
    df_out = pd.DataFrame({'kmer':candidate_kmers, 'specificity':specificity_arr, 'sensitivity':sensitivity_arr,
                           'quality':quality_arr, 'absolute_counts':absolute_counts})
    df_out.to_csv(f'./out_csv/{cluster_name}.csv', index=False)

def greedy_selection(sorted_temp_matrix, clusters_names, clusters_sizes, leaves,
                     sorted_candidate_kmers, local_kmers_of_choice=[], prev_covered_clusters=[],
                     depth=1):
    kmer_occur = -1
    thresh = 0.6
    kmer_of_choice = None
    max_coverage = 0
    max_covered_clusters = []
    for i in sorted_candidate_kmers:
        coverage = 0
        covered_clusters = []
        kmer_occur+=1
        if i in local_kmers_of_choice:
            continue
        cluster_occur = -1
        for cluster in leaves:
            cluster_occur+=1
            if cluster in prev_covered_clusters:
                continue
            if sorted_temp_matrix[cluster_occur][kmer_occur]/clusters_sizes[clusters_names[cluster]] >= thresh:
                coverage+=1
                covered_clusters.append(cluster)
        if coverage > max_coverage:
            max_coverage = coverage
            kmer_of_choice = i
            max_covered_clusters = covered_clusters
    local_kmers_of_choice.append(kmer_of_choice)
    # print('all covered clusters ', prev_covered_clusters)
    prev_covered_clusters.extend(max_covered_clusters)
    print('prev ', len(prev_covered_clusters))
    print('chosen kmers ', len(local_kmers_of_choice))
    print('last kmer ', kmer_of_choice)
    if len(prev_covered_clusters)+max_coverage==len(leaves) or depth == 40:
        prev_covered_clusters.extend(max_covered_clusters)
        print(prev_covered_clusters, len(local_kmers_of_choice))
        return max_covered_clusters, local_kmers_of_choice
    depth+=1
    print('depth ',depth)
    prev_covered_clusters, local_kmers_of_choice = greedy_selection(sorted_temp_matrix, clusters_names, clusters_sizes, leaves,
                                                                    sorted_candidate_kmers, local_kmers_of_choice,
                                                                    prev_covered_clusters, depth) 

def count_uniq(tree, matrix, feature_name, clusters_names, sum_arr, clusters_sizes):
    global leaves
    for k,v in tree.items():
        leaves = []
        get_leaves(v)
        print('len leaves', len(leaves))
        temp_matrix = matrix[leaves]
        clade_sum_arr = np.sum(temp_matrix, axis=0)
        current_clade_size = sum([clusters_sizes[clusters_names[i]] for i in leaves])
        # print('leaves', leaves)
        columns_to_del = np.where(clade_sum_arr < int(0.3*current_clade_size))[0] # фильтрация 60% топовых по чувствительности кмеров
        print('размер клады и threshold',current_clade_size, int(0.3*current_clade_size))
        # удаляем новые отфильтрованные столбцы
        clade_sum_arr = np.delete(clade_sum_arr, columns_to_del)
        temp_sum_arr = np.delete(sum_arr, columns_to_del)
        temp_matrix = np.delete(temp_matrix, columns_to_del, axis=1)
        temp_feature_name = np.delete(feature_name, columns_to_del)

        division_sign = clade_sum_arr/temp_sum_arr
        div_sign_desc = np.argsort(-division_sign)[:200]


        # print(div_sign_desc)
        candidate_kmers = (temp_feature_name[div_sign_desc.tolist()])
        # print(candidate_kmers)
        specificity_arr = (division_sign[div_sign_desc.tolist()])
        print(specificity_arr) # отбор значений чувствительности топовых по чувствительности кмеров
        sensitivity_arr = (clade_sum_arr[div_sign_desc.tolist()]/current_clade_size)
        print(sensitivity_arr)
        absolute_counts = (temp_sum_arr[div_sign_desc.tolist()])
        print(absolute_counts) # абсолютные значения суммы данных кмеров во всем датасете
        local_counts = (clade_sum_arr[div_sign_desc.tolist()])
        sorted_temp_matrix = temp_matrix[:, div_sign_desc.tolist()]
        print(temp_matrix.shape)
        print(sorted_temp_matrix.shape)
        candidate_kmers_sens_sort_order = np.argsort(-sensitivity_arr)
        sorted_candidate_kmers = (candidate_kmers[candidate_kmers_sens_sort_order])
        sorted_temp_matrix = sorted_temp_matrix[:, candidate_kmers_sens_sort_order.tolist()]
        # make_csv(candidate_kmers, specificity_arr, sensitivity_arr, absolute_counts, k)
        local_kmers_of_choice = []
        iterations = 0
        while iterations < 5:
            prev_covered_clusters, local_kmers_of_choice = greedy_selection(sorted_temp_matrix, clusters_names, clusters_sizes, leaves,
                                                                            sorted_candidate_kmers, local_kmers_of_choice, prev_covered_clusters=[])
            iterations+=1
            print('iteration ', iterations)


In [18]:
grp = make_groups(df, diff_column-1)

In [19]:
tree = make_tree(df, grp, diff_column)
tree = tree[list(tree.keys())[0]]

In [20]:
for it in grp.items():
    i = it

In [21]:
print(i)

('Copia', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [22]:
features, feature_name, clusters_names, sum_arr, clusters_sizes =  make_matrix(i)

2101 2101
cluster_temp 14155
cluster_temp_listed 2101
cluster_seq_dict len 2101
500 of 2101
1000 of 2101
1500 of 2101
2000 of 2101
first amount of kmers  6581449
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
kmers after filtering  1402690


In [57]:
leaves = []
count_uniq(tree, features, feature_name, clusters_names, sum_arr, clusters_sizes)

len leaves 238
размер клады и threshold 608 182
[1.         1.         1.         1.         0.99521531 0.99512195
 0.99507389 0.99497487 0.99487179 0.99487179 0.99487179 0.99484536
 0.99484536 0.99470899 0.99468085 0.99141631 0.99115044 0.99107143
 0.99107143 0.99099099 0.99099099 0.99099099 0.99090909 0.99082569
 0.99082569 0.99082569 0.99082569 0.99078341 0.99065421 0.99061033
 0.99061033 0.99052133 0.99052133 0.99052133 0.99047619 0.99029126
 0.9902439  0.99004975 0.99004975 0.98994975 0.98989899 0.98984772
 0.98979592 0.98979592 0.9895288  0.98947368 0.98930481 0.98924731
 0.98913043 0.98828125 0.98818898 0.98790323 0.98785425 0.98780488
 0.98780488 0.98770492 0.98770492 0.98770492 0.98765432 0.98765432
 0.98765432 0.98760331 0.98755187 0.98755187 0.98755187 0.9875
 0.9875     0.9875     0.98739496 0.98734177 0.98734177 0.98728814
 0.98728814 0.98723404 0.98723404 0.98717949 0.98712446 0.98706897
 0.98706897 0.98706897 0.98689956 0.98684211 0.98684211 0.98684211
 0.98678414 0.9867

TypeError: cannot unpack non-iterable NoneType object